## Example 0.5 Get variables for dataset and store in dictionary

In [1]:
from erddapy import ERDDAP

e = ERDDAP(
    server="https://gliders.ioos.us/erddap",
    protocol="tabledap"
)

In [4]:
import functools
import pandas as pd
from erddapy.utilities import urlopen


class Dataset(ERDDAP):
    def __init__(self, server, dataset_id):
        super().__init__(server) 
        self.dataset_id = dataset_id

    @functools.lru_cache(maxsize=None)
    def dataset_attr(self):
        url = self.get_info_url(
            dataset_id=self.dataset_id,
            response="csv",
        )
        df = pd.read_csv(url)
        # The csv info url has two groups: variable and attribute.
        groups = df.groupby("Row Type")
        vs = groups.get_group("variable").dropna(how="all", axis=1)
        variables = {k: f"{dtype}" for k, dtype in zip(vs["Variable Name"], vs["Data Type"])}

        attributes = {}
        attrs = groups.get_group("attribute").dropna(how="all", axis=1).groupby("Variable Name")
        for attr, df in attrs:
            attributes.update(
                {attr: {k: f"{dtype}: {val}" for k, dtype, val in zip(df["Attribute Name"], df["Data Type"], df["Value"])}}
            )
        return variables, attributes
    
    @property
    def attributes(self):
        return self.dataset_attr()[1]

    @property
    def data_variables(self):
        return self.dataset_attr()[0]

In [5]:
ds = Dataset(e.server, dataset_id="whoi_406-20160902T1700")
#ds.attributes
ds.data_variables

{'trajectory': 'String',
 'wmo_id': 'String',
 'profile_id': 'int',
 'time': 'double',
 'latitude': 'double',
 'longitude': 'double',
 'precise_time': 'double',
 'depth': 'float',
 'pressure': 'float',
 'temperature': 'float',
 'conductivity': 'float',
 'salinity': 'float',
 'density': 'float',
 'precise_lat': 'double',
 'precise_lon': 'double',
 'time_uv': 'double',
 'lat_uv': 'double',
 'lon_uv': 'double',
 'u': 'double',
 'v': 'double',
 'platform_meta': 'byte',
 'instrument_ctd': 'byte',
 'precise_lon_qc': 'byte',
 'conductivity_qc': 'byte',
 'temperature_qc': 'byte',
 'precise_time_qc': 'byte',
 'lat_uv_qc': 'byte',
 'density_qc': 'byte',
 'longitude_qc': 'byte',
 'lon_uv_qc': 'byte',
 'time_uv_qc': 'byte',
 'latitude_qc': 'byte',
 'u_qc': 'byte',
 'v_qc': 'byte',
 'depth_qc': 'byte',
 'time_qc': 'byte',
 'pressure_qc': 'byte',
 'precise_lat_qc': 'byte',
 'salinity_qc': 'byte',
 'qartod_temperature_climatological_flag': 'byte',
 'qartod_syntax_flag': 'byte',
 'qartod_salinity_spik